In [1]:
import numpy as np
import random
import shutil

In [2]:
# external custom code I wrote
from load_data import *

Using TensorFlow backend.


In [3]:
[train_paths, val_paths, test_paths], \
[train_waveforms, val_waveforms, test_waveforms], \
[train_procwave, val_procwave, test_procwave], \
[train_wparams, val_wparams, test_wparams], \
[train_windows, val_windows, test_windows] = load_data(TRAIN_SIZE, VAL_SIZE, TEST_SIZE)

In [8]:
NUM_SELECT = 20

test_paths = test_paths[:NUM_SELECT]

In [9]:
base_dir = ""
mturk_dir = base_dir + "mturk-data/"
amr_rates = [8.85, 15.85, 19.85, 23.85]

if not os.path.exists(mturk_dir):
    os.makedirs(mturk_dir)

In [10]:
lead = "    "
print "Preparing data from test set"

# ------------------------------------------------------
# no compression (just highest-quality MP3 conversion)
# ------------------------------------------------------
print lead + "Original files..."

orig_dir = mturk_dir + "orig/"
if not os.path.exists(orig_dir):
    os.makedirs(orig_dir)

for path in test_paths:
    src_file = path

    split = path.split("/")
    identifier = split[-3] + "-" + split[-2] + "-" + split[-1][:-4]
    dest_file = orig_dir + identifier + ".wav"

    ffmpeg_command = "ffmpeg -y -i " + src_file + " " + dest_file
    os.system(ffmpeg_command)

# ------------------------------------------------------
# compress using AMR-WB
# ------------------------------------------------------
for rate in amr_rates:
    print lead + "AMR-WB bitrate " + str(rate) + "..."

    amr_dir = mturk_dir + "amr-" + str(rate) + "/"
    if not os.path.exists(amr_dir):
        os.makedirs(amr_dir)

    for path in test_paths:
        src_file = path

        split = path.split("/")
        identifier = split[-3] + "-" + split[-2] + "-" + split[-1][:-4]
        dest_amr = amr_dir + identifier + ".amr"
        dest_wav = amr_dir + identifier + ".wav"

        command_encode = 'ffmpeg -y -i ' + src_file + ' -ar 16000 -ab ' + str(rate) + 'k ' + \
                         '-acodec libvo_amrwbenc ' + dest_amr
        command_decode = 'ffmpeg -y -i ' + dest_amr + ' ' + dest_wav

        os.system(command_encode)
        os.system(command_decode)
        os.system('rm ' + dest_amr)

Preparing data from test set
    Original files...
    AMR-WB bitrate 8.85...
    AMR-WB bitrate 15.85...
    AMR-WB bitrate 19.85...
    AMR-WB bitrate 23.85...


In [12]:
# prepare input hit file in the format wanted by CrowdMOS
hit = open("MOS.hit_input", "w")
hit.write("sentence\talgorithm\tURL\n")

URL_BASE = "http://srik.zamn.net/mturk-data/"

for path in test_paths:
    src_file = path

    split = path.split("/")
    identifier = split[-3] + "-" + split[-2] + "-" + split[-1][:-4]
    
    # write line for original file
    algo = "orig"
    orig_url = URL_BASE + algo + "/" + identifier + ".wav"
    line = identifier + "\t" + algo + "\t" + orig_url
    hit.write(line + "\n")
    
    # write line for all the AMR-WB rates
    for rate in amr_rates:
        algo = "amr-" + str(rate)
        amr_url = URL_BASE + algo + "/" + identifier + ".wav"
        line = identifier + "\t" + algo + "\t" + amr_url
        hit.write(line + "\n")
    
hit.close()